Assignment 3
1. Chong Kah Wei A20EC0027
2. Heong Yi Qing A20EC0043
3. Mek Zhi Qing A20EC0077
4. Zereen Teo Huey Huey A20EC0173

Grid Search

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# Load data from CSV file
df = pd.read_csv('heart.csv')

# Get the categorical list
catagorialList = ['sex','cp','fbs','restecg','exang','ca','thal']
for item in catagorialList:
    df[item] = df[item].astype('object') #casting to object

# One hot encoding
df = pd.get_dummies(df, drop_first=True)

# Normalization
y = df['target'].values
y = y.reshape(y.shape[0],1)
x = df.drop(['target'],axis=1)
minx = np.min(x)
maxx = np.max(x)
x = (x - minx) / (maxx - minx)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
# re-create train and validation set
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# train 70%, validation 20%, test 10%
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)


(217, 21)
(55, 21)
(31, 21)


In [9]:
# Define the model creation function
def create_model(optimizer='adam', learning_rate=0.001, loss='mse'):
    model = Sequential()
    model.add(Dense(64, input_dim=21, activation='softmax'))
    model.add(Dense(32, activation='softmax'))
    model.add(Dense(1, activation='sigmoid'))

    # Create optimizer instance with the given learning rate
    if optimizer == 'adam':
        optimizer_instance = Adam(learning_rate=learning_rate)
    elif optimizer == 'lion':
        optimizer_instance = tf.keras.optimizers.Lion(learning_rate=learning_rate)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer}")

    model.compile(loss=loss, optimizer=optimizer_instance, metrics=['accuracy'])
    return model

# Wrap the model using KerasClassifier
model = KerasClassifier(model=create_model, verbose=0)

# Define the hyperparameters grid
param_grid = {
    'model__optimizer': ['adam', 'lion'],
    'optimizer__learning_rate': [0.01, 0.001],
    'model__loss': ['binary_crossentropy', 'mse'],
    'epochs': [500, 1000]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print(f"Best Hyperparameters: {grid_result.best_params_}")
print(f"Best Score: {grid_result.best_score_}")

c:\Users\kahwei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kahwei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kahwei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer

Best Hyperparameters: {'epochs': 1000, 'model__loss': 'mse', 'model__optimizer': 'lion', 'optimizer__learning_rate': 0.001}
Best Score: 0.6914003044140031


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

# Performance Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the results
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(confusion_matrix(y_test, y_pred))

Test Accuracy: 0.8387096774193549
Test Precision: 0.9285714285714286
Test Recall: 0.7647058823529411
[[13  1]
 [ 4 13]]


Random Search

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Load data from CSV file
df = pd.read_csv('heart.csv')

# Get the categorical list
catagorialList = ['sex','cp','fbs','restecg','exang','ca','thal']
for item in catagorialList:
    df[item] = df[item].astype('object') #casting to object

# One hot encoding
df = pd.get_dummies(df, drop_first=True)

# Normalization
y = df['target'].values
y = y.reshape(y.shape[0],1)
x = df.drop(['target'],axis=1)
minx = np.min(x)
maxx = np.max(x)
x = (x - minx) / (maxx - minx)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
# re-create train and validation set
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# train 70%, validation 20%, test 10%
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)


(217, 21)
(55, 21)
(31, 21)


In [12]:
# Define the model creation function
def create_model(optimizer='adam', learning_rate=0.001, loss='mse'):
    model = Sequential()
    model.add(Dense(64, input_dim=21, activation='softmax'))
    model.add(Dense(32, activation='softmax'))
    model.add(Dense(1, activation='sigmoid'))

    # Create optimizer instance with the given learning rate
    if optimizer == 'adam':
        optimizer_instance = Adam(learning_rate=learning_rate)
    elif optimizer == 'lion':
        optimizer_instance = tf.keras.optimizers.Lion(learning_rate=learning_rate)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer}")

    model.compile(loss=loss, optimizer=optimizer_instance, metrics=['accuracy'])
    return model

# Wrap the model using KerasClassifier
model = KerasClassifier(model=create_model, verbose=0)

# Define the hyperparameters grid
param_dist = {
    'model__optimizer': ['adam', 'lion'],
    'optimizer__learning_rate': [0.01, 0.001],
    'model__loss': ['binary_crossentropy', 'mse'],
    'epochs': [500, 1000]
}

# Perform random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, scoring='accuracy', cv=3, random_state=42)
random_search_result = random_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print(f"Best Hyperparameters: {random_search_result.best_params_}")
print(f"Best Score: {random_search_result.best_score_}")


c:\Users\kahwei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kahwei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\kahwei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer

Best Hyperparameters: {'optimizer__learning_rate': 0.001, 'model__optimizer': 'lion', 'model__loss': 'binary_crossentropy', 'epochs': 1000}
Best Score: 0.6868975139523085


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
# Evaluate the best model on the test set
best_model = random_search_result.best_estimator_
y_pred = best_model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

# Performance Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the results
print(confusion_matrix(y_test, y_pred))
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")

[[13  1]
 [ 4 13]]
Test Accuracy: 0.8387096774193549
Test Precision: 0.9285714285714286
Test Recall: 0.7647058823529411
